In [26]:
import random                          # to generate random numbers for variability data
from faker import Faker                 # to generate fake but realistic data
from pymongo import MongoClient        # to connect python to MongoDB
from bson.objectid import ObjectId           # to attach unique IDs for MongoDB documents
import pprint as pp                         # for printing query results in readable format
from datetime import datetime,timedelta           # to handle dates
import pandas as pd                           # to analyze and display MongoDB data in table format
from pymongo.errors import PyMongoError              # for error handling
import json                               # to handle JSON data

In [27]:
client = MongoClient("mongodb://localhost:27017/")     # connection to MongoDB
db = client["assignment_db"]
collection = db["employees_collection"]

faker = Faker ()        
departments = ["Engineering", "Marketing", "Customer Service", "Product", "Research", "Finance", "Sales", "HR", "IT Support", "Senior Marketer", "Junior Marketer", "Operations", "Legal", "Design", "Data Science", "Quality Assurance", "Supply Chain", "Business Development"]
positions = ["Manager", "Senior Developer", "Analyst", "Team Lead", "Junior Developer", "Mid-Level Developer", "Software Architect", "Data Engineer", "Data Scientist", "Product Manager", "Project Manager", "Business Analyst", "Marketing Specialist", "SEO Expert", "UI/UX Designer", "HR Manager", "Recruitment Specialist", "Finance Analyst", "Accountant", "Sales Executive", "Customer Support Assistant", "Operations Coordinator", "Legal Advisor", "Supply Chain Manager", "Business Development Executive"]
skills = ["Python", "Java", "JavaScript", "MongoDB", "Docker", "AWS", "SQL", "Excel", "Microsoft Word", "PowerPoint Presentation"]


In [28]:
# Function to insert ONE document into the collection
def insert_document(document):                           
    try:
        result = collection.insert_one(document)  # Insert the document
        return result.inserted_id  # Return the ID of the inserted document
    except Exception as e:
        print(f"Error inserting document: {e}")  # Handle any errors
        return None


# Function to insert MANY documents at once
def insert_many_documents(documents):
    try:
        result = collection.insert_many(documents)  # Insert multiple documents
        return result.inserted_ids  # Return list of inserted IDs
    except Exception as e:
        print(f"Error inserting many documents: {e}")
        return None


# Function to find ONE document
def find_document(query, projection=None):
    try:
        result = collection.find_one(query, projection)  # Find a single matching document
        return result
    except Exception as e:
        print(f"Error finding document:{e}")
        return None


# Function to find MANY documents
def find_many_documents(query, projection=None):
    try:
        result = collection.find(query, projection)  # Find all matching documents
        return result
    except Exception as e:
        print(f"Error finding many documents: {e}")
        return None


# Function to UPDATE ONE document
def update_document(query, update):
    try:
        result = collection.update_one(query, update)  # Update first matching document
        return result.modified_count  # Return number of documents updated
    except Exception as e:
        print(f"Error updating document: {e}")
        return None


# Function to UPDATE MANY documents
def update_many_documents(query, update):
    try:
        result = collection.update_many(query, update)  # Update all matching documents
        return result.modified_count  # Return number of documents updated
    except Exception as e:
        print(f"Error updating many documents: {e}")
        return None


# Function to DELETE ONE document
def delete_document(query):
    try:
        result = collection.delete_one(query)  # Delete first matching document
        return result.deleted_count  # Return number of deleted documents
    except Exception as e:
        print(f"Error deleting document: {e}")
        return None


# Function to DELETE MANY documents
def delete_many_documents(query):
    try:
        result = collection.delete_many(query)  # Delete all matching documents
        return result.deleted_count  # Return number of deleted documents
    except Exception as e:
        print(f"Error deleting many documents: {e}")
        return None


# Function to COUNT documents
def count_documents(query):
    try:
        result = collection.count_documents(query)  # Count number of matching documents
        return result
    except Exception as e:
        print(f"Error counting documents: {e}")
        return None


# (Optional) Function to print all employees
# def show_employees():    
#     for emp in collection.find(): 
#         print(emp)


In [29]:

sample_employees = []

# Generate fake employees
for i in range(10000):
    employee = {
        "_id": ObjectId(),
        "employee_id": f"EMP{str(i+2).zfill(3)}",
        "first_name": faker.first_name(),
        "last_name": faker.last_name(),
        "email": faker.email(),
        "department": random.choice(departments),
        "position": random.choice(positions),
        "salary": random.randint(40000, 120000),
        "years_experience": random.randint(1, 15),
        "performance_rating": round(random.uniform(1.0, 5.0), 1),
        "skills": random.sample(skills, k=random.randint(1, 5)),  # random set of skills
        "hire_date": faker.date_time_between(start_date="-5y", end_date="now"),
        "last_promotion": faker.date_time_between(start_date="-2y", end_date="now"),
        "is_remote": random.choice([True, False]),
        "address": {
            "city": faker.city(),
            "state": faker.state_abbr(),
            "zip_code": faker.zipcode()
        }
    }
    sample_employees.append(employee)    #  

# Insert into Mongo
collection.delete_many({})  # clear old data
collection.insert_many(sample_employees)
print(f"Inserted {len(sample_employees)} employees into MongoDB.")


Inserted 10000 employees into MongoDB.


In [30]:
count = collection.count_documents({})                 # Count total documents in the collection
print(f"Total documents in collection: {count}")

Total documents in collection: 10000


In [33]:

def get_high_performers(col):
   
    try:
        query = {
            "performance_rating": {"$gte": 4.0},  # >= 4.0
            "salary": {"$gt": 80000}              # > 80,000
        }
        projection = {
            "_id": 0,
            "first_name": 1,
            "last_name": 1,
            "department": 1,
            "salary": 1,
            "performance_rating": 1
        }
        data = list(col.find(query, projection).limit(3))
        return pd.DataFrame(data)
    except PyMongoError as e:
        print(f"Database error: {e}")
        return pd.DataFrame()



"""
    Find employees with 5-10 years experience AND salary between $70k-$120k.
    Projection returns contact info.
    """
# 2. Experience-Based Filtering
def filter_by_experience_and_salary(col):
    
    try:
        query = {
            "years_experience": {"$gte": 5, "$lte": 10},     # between 5 and 10 years
            "salary": {"$gte": 70000, "$lte": 120000}        # between $70k and $120k
        }
        projection = {
            "_id": 0,
            "first_name": 1,
            "last_name": 1,
            "email": 1,
            "department": 1
        }
        data = list(col.find(query, projection).limit(3))
        return pd.DataFrame(data)
    except PyMongoError as e:
        print(f"Database error: {e}")
        return pd.DataFrame()

"""
    Find employees whose salary is NOT between $60k and $100k.
    Projection returns full name, salary, and years of experience.
    """
# Salary Range Analysis
def salary_outside_range(col):
   
    try:
        query = {
            "salary": {"$not": {"$gte": 60000, "$lte": 100000}}  # NOT in range
        }
        projection = {
            "_id": 0,
            "full_name": {"$concat": ["$first_name", " ", "$last_name"]},  # needs aggregation
            "salary": 1,
            "years_experience": 1
        }
        # Must use aggregation to create full_name
        data = list(col.aggregate([
            {"$match": query},
            {"$project": {
                "_id": 0,
                "full_name": {"$concat": ["$first_name", " ", "$last_name"]},
                "salary": 1,
                "years_experience": 1
            }},
            {"$limit": 3}
        ]))
        return pd.DataFrame(data)
    except PyMongoError as e:
        print(f"Database error: {e}")
        return pd.DataFrame()

"""
    Find employees hired in the last 2 years with performance > 3.5.
    Return full_name, tenure_months, and annual_salary.
    """
#  Recent Hires  - query
def recent_hires(col):
    
    try:
        two_years_ago = datetime.now() - timedelta(days=730)
        query = {
            "hire_date": {"$gte": two_years_ago},
            "performance_rating": {"$gt": 3.5}
        }
        data = list(col.aggregate([
            {"$match": query},
            {"$project": {
                "_id": 0,
                "full_name": {"$concat": ["$first_name", " ", "$last_name"]},
                "tenure_months": {
                    "$divide": [
                        {"$subtract": [datetime.now(), "$hire_date"]},
                        1000 * 60 * 60 * 24 * 30
                    ]
                },
                "annual_salary": "$salary"
            }},
            {"$limit": 3}
        ]))
        return pd.DataFrame(data)
    except PyMongoError as e:
        print(f"Database error: {e}")
        return pd.DataFrame()

high_performers = get_high_performers(collection)
experience_salary = filter_by_experience_and_salary(collection)
outside_salary = salary_outside_range(collection)
recent = recent_hires(collection)

# 1. High performers
print("High Performers:")
print(get_high_performers(collection))

# 2. Experience filter
print("\nEmployees with 5–10 yrs experience and salary 70k–120k:")
print(filter_by_experience_and_salary(collection))

# 3. Salary outside range
print("\nEmployees with salary NOT between 60k–100k:")
print(salary_outside_range(collection))

# 4. Recent hires
print("\nRecent hires (last 2 years, perf > 3.5):")
print(recent_hires(collection))


results = {
    "High Performers": high_performers.to_dict(orient="records"),
    "Experience & Salary Filter": experience_salary.to_dict(orient="records"),
    "Salary Outside Range": outside_salary.to_dict(orient="records"),
    "Recent Hires": recent.to_dict(orient="records")
}

# Save to JSON file
with open("employee_results.json", "w") as f:
    json.dump(results, f, indent=4, default=str) 


High Performers:
    first_name last_name         department  salary  performance_rating
0  Christopher  Robinson          Marketing   94189                 4.8
1    Alexandra     Moore  Quality Assurance  112637                 4.9
2         Rick   Johnson    Junior Marketer  115300                 4.9

Employees with 5–10 yrs experience and salary 70k–120k:
    first_name  last_name                      email            department
0        David  Hernandez   johnsoncarol@example.com                 Sales
1          Roy      Nunez     davidsmith@example.net  Business Development
2  Christopher   Robinson  fernandezjudy@example.net             Marketing

Employees with salary NOT between 60k–100k:
   salary  years_experience       full_name
0   56554                15  Richard Taylor
1   47775                 4     Anne Norris
2  102516                 1    Allison Lowe

Recent hires (last 2 years, perf > 3.5):
        full_name  tenure_months  annual_salary
0    Rick Johnson       4.2